# PyVesTo

Python Blood Vessel Topology Analysis (PyVesTo) is a framework for analysing blood vessel digital images. This includes the segmentation, representation and characterization of blood vessels. The framework identifies 2D and 3D vascular systems and represent them using graphs. The graphs describe the topology of the blood vessels, that is, bifurcations and terminations are represented as nodes and two nodes are connected if there is a blood vessel segment between them.

# Examples of PyVesTo usage
## Using the default pipeline to perform blood vessel morphometry

In [1]:
import sys
import os
from pathlib import Path
from functools import partial
import numpy as np
import networkx as nx
path = Path(os.path.abspath(''))
sys.path.append(str(path.parents[0]))

from pyvesto import pipeline, image

Some important parameters:
* ***channel (int)***: Channel to be processed. If None, all image data will be considered.
* ***name_filter (function)***: Here you can pass a function for filtering files. This function should return True if the file should be processed, and False otherwise. If name_filter == None, all image files will be processed.
* ***save_steps (tuple of str)***: A tuple of strings containing which intermediate data should be saved on disk.  The default behaviour (save_steps == 'all') is to save all intermediate steps: ('segmentation', 'skeletonization', 'network', 'analysis').
* ***start_at (int)***: Given the list of images to be processed, start at this index. If 0 or None, all images will be processed.
* ***verbosity (int)***: Level of verbosity (0, 1, 2, 3).

In [2]:
channel = None
name_filter = None
save_steps = 'all'
start_at = 0
verbosity = 3

The ***img_reader***, is the function used to read the image given a filename. The default image reader used in PyVesTo is located in ***pipeline.read_and_adjust_img***. This function receives a filename (Path or str) and reads the image using the ***img_io*** utility functions. After reading, the data is transformed to float, the linear transformation \[min, max\] -> \[0, 255\] is applied, and finally the image is interpolated to make it isotropic. The function returns an ***image.Image*** object.

In [3]:
img_reader = partial(pipeline.read_and_adjust_img, channel=channel)

A PyVesTo morphometry pipeline is built through the class ***pipeline.BasePipeline***. Here, three parameters are needed:
* ***input_path***: Folder containing the input images (Path or str)
* ***output_path***: Folder to save images generated from intermediate steps (Path or str)
* ***img_reader***: The image reader that will be used.

In [4]:
input_path = Path('../data/images')
output_path = Path('../data/results')

dp = pipeline.BasePipeline(input_path, img_reader, output_path=output_path, 
                           name_filter=name_filter, save_steps=save_steps, start_at=start_at, verbosity=verbosity)

PyVesTo works with a sequence of subroutines that we call **processors**. Each processor performs a distinct step in the morphometry pipeline. In total, four processors are required: A segmenter, a skeleton builder, a network builder, and an analyzer. PyVesTo already has implemented four default processors that can be used in the aforementioned steps. These processors comprises the methodologies used in recent publications (listed [here](https://github.com/chcomin/pyvesto)).

In [5]:
threshold = 0
sigma = None

segmenter = pipeline.DefaultSegmenter(threshold, sigma)
skeleton_builder = pipeline.DefaultSkeletonBuilder()
network_builder = pipeline.DefaultNetworkBuilder()
analyzer = pipeline.DefaultAnalyzer(10)

Defined the processors that will be used, they should be added to the BasePipeline object through ***pipeline.BasePipeline.set_processors***.

In [6]:
dp.set_processors(segmenter, skeleton_builder, network_builder, analyzer)

After these steps, the whole pipeline can be executed through ***pipeline.BasePipeline.run***. The obtained results will be saved on the ***output_path*** folder.

In [7]:
dp.run()

Processing file AVG_40 S1 layer IV (1 of 1)...


{'AVG_40 S1 layer IV': {'Length (mm/mm^3)': 62.93855587801143,
  'Branching points (1/mm^3)': 2204.589020896528,
  'Tortuosity': 0.29562741674406673}}

## Building your own processors

The main reason PyVesTo breaks the morphometry pipeline into four processors, is to able the user writing its own processor as needed. Therefore, one can write a custom segmenter and still use the rest of our default pipeline.

All PyVesTo processors should inherit the class ***pipeline.BaseProcessor***. This class implement a \_\_call\_\_() method that calls ***apply()*** when a ***pipeline.BaseProcessor*** instance is called as a function. Therefore a custom processor can be writen inheriting ***pipeline.BaseProcessor*** and overriding the ***apply()*** method. For example:

In [8]:
import scipy.ndimage as ndi

class CustomSegmenter(pipeline.BaseProcessor):
    def __init__(self, threshold=0, radius=40):
        # Initialize your variables as you need
        self.threshold = threshold
        self.radius = radius
    
    def apply(self, img, file=None):
        # Your segmentation code goes here
        
        # This is a much simpler version of segmentation.vessel_segmentation
        # for explaining purposes
        
        img_data = img.data.astype(np.float)
        img_blurred = ndi.gaussian_filter(img_data, sigma=self.radius/2.)
        img_corr = img_data - img_blurred
        img_bin = img_corr > self.threshold
        img_bin = image.Image(img_bin, path=img.path, pix_size=img.pix_size)
        
        return img_bin

Then, we can define a BasePipeline with the same parameters as before, only changing the segmenter.

In [9]:
custom_p = pipeline.BasePipeline(input_path, img_reader, output_path=output_path, name_filter=name_filter, 
                          start_at=start_at, verbosity=verbosity)

In [10]:
segmenter = CustomSegmenter()
skeleton_builder = pipeline.DefaultSkeletonBuilder()
network_builder = pipeline.DefaultNetworkBuilder()
analyzer = pipeline.DefaultAnalyzer(10)

custom_p.set_processors(segmenter, skeleton_builder, network_builder, analyzer)

In [11]:
custom_p.run()

Processing file AVG_40 S1 layer IV (1 of 1)...


{'AVG_40 S1 layer IV': {'Length (mm/mm^3)': 70.63979558452819,
  'Branching points (1/mm^3)': 2834.5656386922815,
  'Tortuosity': 0.3526943147370302}}